In [1]:
import os

import rich

from mdmodels.sql import DatabaseConnector, generate_sqlmodel, select


In [2]:
if os.path.exists("test.db"):
    os.remove("test.db")

In [3]:
models = generate_sqlmodel(path="model.md")
db = DatabaseConnector(database="test.db")

db.create_tables(models)

In [4]:
# Extract tables to create rows
ChemicalProject = models["ChemicalProject"]
Molecule = models["Molecule"]

In [5]:
with db as session:
    # Case 1: Insert a project with a molecule
    project = ChemicalProject(title="Test Project")
    project.molecules.append(
        Molecule(id="mol1", name="Methanol", smiles="CO")
    )
    
    session.add(project)

In [6]:
with db as session:
    # Case 2: Extract the project and print it
    statement = select(ChemicalProject).where(ChemicalProject.title == "Test Project")
    projects = session.exec(statement).all()
    
    for project in projects:
        rich.print(project.to_dict())

{
    'title': 'Test Project',
    'molecules': [{'id': 'mol1', 'name': 'Methanol', 'formula': None}],
    'reactions': [],
    'experiments': []
}

In [7]:
with db as session:
    # Case 3: Add a molecule to the database and link it to the project
    molecule = Molecule(id="mol2", name="Ethanol", smiles="CCO")
    session.add(molecule)
    
    # Add the molecule to the project
    statement = select(ChemicalProject).where(ChemicalProject.title == "Test Project")
    project = session.exec(statement).first()
    
    project.molecules.append(molecule)
    session.add(project)

In [8]:
with db as session:
    # Case 4: Update the molecule
    statement = select(Molecule).where(Molecule.id == "mol2")
    molecule = session.exec(statement).first()
    
    molecule.name = "Ethanol (Updated)"
    molecule.formula = "C2H6O"
    
    session.add(molecule)

In [9]:
with db as session:
    # Verify the molecule was added to the project and updated
    statement = select(ChemicalProject).where(ChemicalProject.title == "Test Project")
    project = session.exec(statement).first()
    
    rich.print(project.to_dict())

{
    'title': 'Test Project',
    'molecules': [
        {'id': 'mol1', 'name': 'Methanol', 'formula': None},
        {'id': 'mol2', 'name': 'Ethanol (Updated)', 'formula': 'C2H6O'}
    ],
    'reactions': [],
    'experiments': []
}